# Ask Extension data EDA

In [7]:
import pandas as pd

import json
import os

from string import punctuation as pn

In [8]:
PATH = '../data/askextension/2020-08-20/'
FILE_NAMES = [PATH + f for f in sorted(os.listdir(PATH))]

## Data

Data constists 4 files:
- 2012-2014.json
- 2014-2016.json
- 2016-2018.json
- 2018-2020.json

And looks following way:

In [9]:
with open(FILE_NAMES[0]) as f:
    f = json.load(f)
    print(json.dumps(f[0], indent = 2))

{
  "faq-id": 3,
  "title": "When can I plant blue spruce trees in Colorado? #109900",
  "created": "2012-12-03 15:53:47",
  "updated": "2012-12-03 17:47:21",
  "tags": [
    "trees and shrubs"
  ],
  "state": "Colorado",
  "county": "El Paso County",
  "question": "I need to plant two blue spruce trees that are currently in 24\" diameter plastic containers with drain holes in the bottom sides.\n\nLocation: northeast side of Colorado Springs.\n\nThese trees are currently outside on the patio and susceptible to the wind and sun. The trees were watered this past Saturday and seem to be healthy.\n\nQuestion: Can these trees be planted now? Currently the soil is not frozen and night time temps are 35 to 40 degrees.\n\nI have downloaded and read CMG GardenNotes #633 as a reference.\n\nAny advice would be greatly appreciated. ",
  "answer": {
    "1": {
      "response": "Jerry, \nyou can plant them now (a) OR temporarily \"plant\" them, still in containers, so that roots have some insulatio

It is a list of dictionary objects with following fields:
- `faq-id` - ID of the ticket
- `title` - title of the ticket along ID of the ticket (__other__ ID)
- `created` - ticket creating date
- `updated` - ticket last update date
- `tags` - list of tags
- `state` - state ticket was created in
- `county` - county ticket was created in
- `question` - question that has been posted
- `answer` - response lists presented in numbered dictionary data type

## ETL

Following transformations are performed:
- merge source into single JSON file
- filter out ticket from all states except _California_
- retrive ticket ID from title
- create URL of ticket
- clean long text columns - `title`, `question`, `answer`
- create new column out of `title` and `question`
- remove tickets that have less than 3 words in `title` and `quesiton` combined
- limit the number of characters to 300 for `title`, `question`, `answer` columns

Save result to ES JSON format to be injected to ES.

## EDA

TO BE DONE

In [10]:
import config

In [14]:
pd.read_json(config.ASKEXTENSION_FILE_RESULT).set_index('index')

,faq-id,ticket-no,url,created,updated,state,county,title,question,title-question,answer
index,,,,,,,,,,,
0,93908,437288,https://ask2.extension.org/kb/faq.php?id=437288,2018-01-02 12:17:46,2018-01-04 18:39:35,California,Monterey County,Best coop for backyard chickens.,Can you suggest a coop for my backyard Id like...,Best coop for backyard chickens. Can you sugge...,"[There is no ""best"" chicken coop. When housing..."
1,93903,437706,https://ask2.extension.org/kb/faq.php?id=437706,2018-01-09 19:01:42,2018-01-09 19:46:37,California,Los Angeles County,Small black bugs with little white colored wings.,Woke up this morning and found these small ins...,Small black bugs with little white colored win...,[I cannot identify the insect from the picture...
2,93889,437799,https://ask2.extension.org/kb/faq.php?id=437799,2018-01-11 03:52:59,2018-01-11 15:37:51,California,Los Angeles County,Browning Sequoia Sappling.,Hello! I have a sequoia sappling that has some...,Browning Sequoia Sappling. Hello! I have a seq...,[Thank you for your sequioa question. Browning...
3,93987,437923,https://ask2.extension.org/kb/faq.php?id=437923,2018-01-13 01:33:18,2018-01-16 00:21:06,California,Siskiyou County,What type of snake is this?,"Hello, I was recently out walking near the cre...","What type of snake is this? Hello, I was recen...",[Thank you for your question. The snake in you...
4,93840,438008,https://ask2.extension.org/kb/faq.php?id=438008,2018-01-14 20:48:29,2018-01-21 01:02:27,California,Los Angeles County,Name this spider.,I have this awesome spider necklace that I got...,Name this spider. I have this awesome spider n...,"[Dear Client, I am not certain what species of..."
...,...,...,...,...,...,...,...,...,...,...,...
2166,93487,436235,https://ask2.extension.org/kb/faq.php?id=436235,2017-12-10 17:27:31,2017-12-14 17:13:20,California,Lincoln County,Tulip.,I live in California and I have a few bags of ...,Tulip. I live in California and I have a few b...,[Thank you for your tulip question. Tulip bulb...
2167,93489,436359,https://ask2.extension.org/kb/faq.php?id=436359,2017-12-12 18:06:08,2017-12-13 19:59:10,California,Solano County,Wolf or Recluse.,"Someone please help me, is this a wolf spider ...","Wolf or Recluse. Someone please help me, is th...","[This is not a brown recluse spider, it does a..."
2168,93833,436669,https://ask2.extension.org/kb/faq.php?id=436669,2017-12-19 05:57:02,2017-12-22 22:47:51,California,Alameda County,Do spiders stay warm at night?,I have a weather station that is mounted to my...,Do spiders stay warm at night? I have a weathe...,[The body of spiders (and other invertebrates)...
